In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection

# Note: to use OptPiecewiseRegression, a working pyomo and cplex optimizer is needed.
from MyPiecewiseRegression import OptPiecewiseRegression, LSPPiecewiseRegression

# The solver to be used throughout the script.
from pyomo.environ import SolverFactory
opt = SolverFactory("cplex")

# Random seed for splitting the data in validation
random_state=3456787897

In [2]:
# Preprocess the data by dropping the Newspaper column,

# Two outliers (id: 5, 130) were detected in previous work.
# Uncomment the .drop line below to exclude these two outliers.

df = (
    pd.read_csv("Advertising1.csv")
    .drop(columns=["Unnamed: 0", "Newspaper"])
#    .drop([5, 130])
)
df.head()

,TV,Radio,Sales
0,230.1,37.8,22.1
1,44.5,39.3,10.4
2,17.2,45.9,9.3
3,151.5,41.3,18.5
4,180.8,10.8,12.9


In [3]:
# An alternative way to detect the outliers is to
# check the Mahalanobis distance between each data
# point and the mean.

def mahalanobis(X: pd.DataFrame):
    X0 = np.array(X - X.mean())
    S_inv = np.linalg.inv(X.cov())
    
    return np.sqrt(np.sum(X0.dot(S_inv) * X0, axis=1))

np.where(mahalanobis(df[["TV", "Radio"]]) > 2)

(array([  2,   5,   8,  35,  75,  98, 108, 130, 135, 147, 175, 178, 183],
       dtype=int64),)

In [4]:
# We add a constant of one into the dataset to represent the intercept.
X = df.loc[:, ["TV", "Radio"]]
X["const"] = 1.0
X = X.loc[:, ["const", "TV", "Radio"]].to_numpy()
y = df.loc[:, "Sales"].to_numpy()

## Regression Part (Section 8.1)

In [5]:
# Piecewise Linear Fitting
# Fitting a convex model on concave data is slow.
# OptPiecewiseRegression solves the regression problem exactly by a MIQP,
# and LSPPiecewiseRegression is a heuristic using Least Squares Partition.
# To fit a 3-piece model, it is advised to warm start it with a heuristic.
# After fitting, the coef_ attribute is set: np.array of shape (n, d)
# with n <= n_pieces, each vector of size (d, ) represents an hyperplane.

# Example usage:
# reg = OptPiecewiseRegression(convex=True, n_pieces=2, opt=opt)
# reg.fit(X, y)
# print(reg.coef_)


regressors = {
#    "convex_2_piecewise": OptPiecewiseRegression(convex=True, n_pieces=2, opt=opt),
    "concave_2_piecewise": OptPiecewiseRegression(convex=False, n_pieces=2, opt=opt),
    "concave_2_piecewise_heuristic": LSPPiecewiseRegression(convex=False, n_pieces=2),
    "concave_3_piecewise_heuristic": LSPPiecewiseRegression(convex=False, n_pieces=3),
    "linear": LinearRegression(fit_intercept=False)
}

In [6]:
# Now we fit the regressors and summarize the results.
# This may take a while.
for reg_name, reg in regressors.items():
    print("Fitting {}".format(reg_name))
    reg.fit(X, y)
print("Done")

reg_results = pd.DataFrame(
    [
        [
            reg_name,
            r2_score(y, reg.predict(X)),
            mean_squared_error(y, reg.predict(X)),
            reg.coef_.round(3)
        ]
        for reg_name, reg in regressors.items()
    ],
    columns = ["Regressor", "R-squared", "MSE", "coef"]
)
reg_results

Fitting concave_2_piecewise
Fitting concave_2_piecewise_heuristic
Fitting concave_3_piecewise_heuristic
Fitting linear
Done


,Regressor,R-squared,MSE,coef
0,concave_2_piecewise,0.976671,0.631885,"[[4.027, 0.077, 0.074], [5.44, 0.025, 0.273]]"
1,concave_2_piecewise_heuristic,0.976671,0.631885,"[[4.027, 0.077, 0.074], [5.44, 0.025, 0.273]]"
2,concave_3_piecewise_heuristic,0.985568,0.390890,"[[0.637, 0.534, 0.032], [4.172, 0.069, 0.095],..."
3,linear,0.897194,2.784570,"[2.921, 0.046, 0.188]"


## Decision models (Section 8.2)

In [7]:
# This contains the various decision models for the LEO-Wyndor problem.
import DecisionModels

In [8]:
# Here we reuse the concave piecewise linear regression model
# and feed the coefficients to the corresponding decision model.
reg = regressors["concave_2_piecewise"]
resid = y - reg.predict(X)

# This returns a pyomo model. Only the concave case is implemented.
model = DecisionModels.get_piecewise_model(reg.coef_, resid)
opt.solve(model)
print("Objective:", model.obj())
print("Decision: ", [model.x1(), model.x2()])

Objective: -45.0043676277769
Decision:  [123.2248930987497, 25.08211659343577]


## Validation (Table "Solution ... with Concave Piecewise Regression")

In [9]:
# This defines the validation framework.
# reg_model: the regression model. A class with fit() predict() and attribute coef_;
# this specifies how the model produces a fit for the data. e.g. LinearRegression.
# opt_model: a function (reg_model, resid) -> (mpo, decision)
# which specifies how a decision is made given the fitted regression model
# and the residuals. See examples in the next cell.
# If opt_model is None, the function will quit after fitting.
# sec_stage_model: a function (x, reg_model, resid) -> sec_obj.
# This specifies how the objective is evaluated given a decision and regression model.
# splitter: the sklearn.model_selection.KFold object used to split the dataset.

# This produces a list of dictionaries,
# each dictionary containing the result for one split.
# The valid keys for dictionary is:
# ['Fitting MSE', 'Model Reported Objective', 'Decision',
# 'Training Objective', 'Validation Objective',
# 'Training Index', 'Validation Index']

# Example usage:
# result = validate_pw_model(
#     splitter=kfold,
#     reg_model=OptPiecewiseRegression(convex=False, n_pieces=2, opt=opt),
#     opt_model=PW_SAA_opt_model,
#     sec_stage_model=PW_second_stage
# )
# print("validation mean:", np.mean(result[0]["Validation Objective"]))
# print("mpo:", result[0]["Model Reported Objective"])

def validate_pw_model(reg_model, opt_model, sec_stage_model, splitter, verbose=False):
    logs = []

    for train_index, val_index in splitter.split(X, y):
        split_log = {}

        X_train = X[train_index]
        y_train = y[train_index]
        X_val = X[val_index]
        y_val = y[val_index]

        # Fitting
        reg = reg_model.fit(X_train, y_train)

        train_resid = y_train - reg.predict(X_train)
        val_resid = y_val - reg.predict(X_val)
        split_log["Fitting MSE"] = [(train_resid ** 2).mean(), (val_resid ** 2).mean()]
        
        # Print training residue if needed for external program
        if verbose:
            print(train_resid)
            np.save("train_resid.npy", train_resid)
        
        if opt_model is None:
            return
        
        # Optimization
        model_objective, x = opt_model(reg, train_resid)
        split_log["Model Reported Objective"] = model_objective
        split_log["Decision"] = x.tolist()
        
        # Validation (freeze first stage decision)
        train_obj = []
        for res in train_resid:
            train_obj.append(sec_stage_model(x, reg, res) + [0.1, 0.5] @ x)
        split_log["Training Objective"] = train_obj
        
        val_obj = []
        for res in val_resid:
            val_obj.append(sec_stage_model(x, reg, res) + [0.1, 0.5] @ x)
        split_log["Validation Objective"] = val_obj
        
        split_log["Training Index"] = train_index
        split_log["Validation Index"] = val_index

        logs.append(split_log)
    
    return logs

In [10]:
# The definition of optimization models

# Deterministic model with linear regression only (DF/LP)
# Should be used with LinearRegession
def DF_LP_opt_model(reg, resid):
    model = DecisionModels.get_deterministic_model(reg.coef_, 0)
    opt.solve(model)
    return model.obj(), np.array([model.x1(), model.x2()])

# SAA model with linear regression (EAE/SAA)
# Should be used with LinearRegression
def EAE_SAA_opt_model(reg, resid):
    model = DecisionModels.get_all_in_one_model(reg.coef_, resid)
    opt.solve(model)
    return model.obj(), np.array([model.x1(), model.x2()])

# SAA model with concave piecewise linear regression
# Should be used with OptPiecewiseRegression or LSPPiecewiseRegression
def PW_SAA_opt_model(reg, resid):
    model = DecisionModels.get_piecewise_model(reg.coef_, resid)
    opt.solve(model)
    return model.obj(), np.array([model.x1(), model.x2()])

# To speed up computations, we use the following equivalent second stage functions
# This can be obtained for some problems using ranging/sensitivity analysis on
# the second stage LP's.
@np.vectorize
def profit_fn(omega):
    if omega < 0:
        return -np.inf
    elif omega <= 12:
        return 5.0*omega
    elif omega <= 16:
        return 60.0+3*(omega-12)
    else:
        return 72.0

# These defines the second stage functions h(x, omega)
# where omega = reg.predict([x]) + res
# reg is LinearRegression for LP_second_stage, and it is
# one of the PiecewiseRegresion for PW_second_stage
def LP_second_stage(x, reg, res):
    return -profit_fn(
        reg.predict(np.array([[1, x[0], x[1]]])) + res
    ).item()

def PW_second_stage(x, reg, res):
    return -profit_fn(
        reg.predict(np.array([[1, x[0], x[1]]])) + res
    ).item()

# or we can use the original definitions below, if preferred.

# def LP_second_stage(x, reg, res):
#     sec_model = DecisionModels.get_second_stage_model(x, reg.coef_, res)
#     opt.solve(sec_model)
#     return sec_model.obj()

# def PW_second_stage(x, reg, res):
#     sec_model = DecisionModels.get_second_stage_piecewise_model(x, reg.coef_, res)
#     opt.solve(sec_model)
#     return sec_model.obj()

In [11]:
# Here are some possible combinations of regression models and decision models.
methods = {
    "DF_LP": {
        "reg_model": LinearRegression(fit_intercept=False),
        "opt_model": DF_LP_opt_model,
        "sec_stage_model": LP_second_stage
    },
    "EAE_SAA": {
        "reg_model": LinearRegression(fit_intercept=False),
        "opt_model": EAE_SAA_opt_model,
        "sec_stage_model": LP_second_stage
    },
    # EAE/SD is a little trickier: we need to dump the data out and
    # run SD Solver and manually input the decision (compromise solution)
    "EAE_SD": {
        "reg_model": LinearRegression(fit_intercept=False),
        "opt_model":
        lambda reg, resid : (-39.829668, np.array((183.129419, 16.870581))), 
        "sec_stage_model": LP_second_stage
    },
    "PW2_SAA": {
        "reg_model": OptPiecewiseRegression(n_pieces=2, convex=False, opt=opt),
        "opt_model": PW_SAA_opt_model,
        "sec_stage_model": PW_second_stage
    },
    "PW3_SAA": {
        "reg_model": LSPPiecewiseRegression(n_pieces=3, convex=False),
        "opt_model": PW_SAA_opt_model,
        "sec_stage_model": PW_second_stage        
    }
}

# To specify a fixed decision, use the following code.

# fixed_decision = {
#     "reg_model": LinearRegression(fit_intercept=False),
#     "opt_model": lambda reg, resid: (-39.887, np.array([181.629, 18.370])),
#     "sec_stage_model": LP_second_stage
# }



In [12]:
# Now instantiate the split and run the validation process.
results = {}
kfold = model_selection.KFold(n_splits=2, shuffle=True, random_state=random_state)

for method in methods.keys():
    print("Evaluating {}".format(method))
    results[method] = validate_pw_model(splitter=kfold, **methods[method])
print("Done")

Evaluating DF_LP
Evaluating EAE_SAA
Evaluating EAE_SD
Evaluating PW2_SAA
Evaluating PW3_SAA
Done


In [13]:
# Summarize the results into a table.
# Note that the results are scaled by -1000
# to convert the unit from negative thousand
# dollars to dollars.

split_index = 0
N_v = len(X) / 2

val_results = pd.DataFrame(
    [
        [
            method,
            result[split_index]["Decision"],
            -1000*result[split_index]["Model Reported Objective"],
            -1000*np.mean(result[split_index]["Validation Objective"]),
            1000*2*np.std(result[split_index]["Validation Objective"])/np.sqrt(N_v)
        ]
        for method, result in results.items()
    ],
    columns=["Methodology", "Decision", "MPO", "MVSAE", "MVSAE half-width"]
)

val_results["MVSAE CI lower"] = val_results["MVSAE"] - val_results["MVSAE half-width"]
val_results["MVSAE CI upper"] = val_results["MVSAE"] + val_results["MVSAE half-width"]

val_results

,Methodology,Decision,MPO,MVSAE,MVSAE half-width,MVSAE CI lower,MVSAE CI upper
0,DF_LP,"[172.50058130739086, 27.49941869260914]",41000.232523,39278.453787,623.056607,38655.397180,39901.510394
1,EAE_SAA,"[184.12416941917957, 15.875830580820434]",40212.812449,40733.326771,995.200173,39738.126597,41728.526944
2,EAE_SD,"[183.129419, 16.870581]",39829.668000,40687.522266,964.167195,39723.355071,41651.689461
3,PW2_SAA,"[120.49049755454976, 25.849837330023714]",44849.907766,45213.914593,356.288897,44857.625696,45570.203490
4,PW3_SAA,"[134.41045334868525, 24.511296843800487]",44646.143789,44722.408369,305.363547,44417.044822,45027.771915
